## Creating DDL 

In [52]:
import sqlite3

# Crear la conexión a la base de datos
conn = sqlite3.connect('renta_vehiculos.db')

# Crear tabla de vehículos
conn.execute('''CREATE TABLE IF NOT EXISTS vehiculos (
    id INTEGER PRIMARY KEY,
    marca TEXT,
    modelo TEXT,
    anio INTEGER,
    tipo TEXT,
    capacidad INTEGER,
    color TEXT,
    kilometraje REAL,
    estado TEXT
)''')

# Crear tabla de puntos de renta
conn.execute('''CREATE TABLE  IF NOT EXISTS puntos_renta (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    direccion TEXT,
    telefono TEXT,
    horario_atencion TEXT
)''')

# Crear tabla de clientes
conn.execute('''CREATE TABLE IF NOT EXISTS clientes (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    correo_electronico TEXT,
    telefono TEXT,
    direccion TEXT
)''')

# Crear tabla de rentas
conn.execute('''CREATE TABLE IF NOT EXISTS rentas (
    id INTEGER PRIMARY KEY,
    id_vehiculo INTEGER,
    id_punto_salida INTEGER,
    id_punto_retorno INTEGER,
    id_cliente INTEGER,
    fecha_inicio TEXT,
    fecha_fin TEXT,
    costo_total REAL,
    duracion_renta TEXT,
    FOREIGN KEY (id_vehiculo) REFERENCES vehiculos(id_vehiculo),
    FOREIGN KEY (id_punto_salida) REFERENCES puntos_renta(id_punto),
    FOREIGN KEY (id_punto_retorno) REFERENCES puntos_renta(id_punto),
    FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente)
)''')

# Extraer datos relevantes de las tablas de vehículos y rentas
query = '''SELECT v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad, COUNT(*) AS num_rentas
FROM vehiculos v
INNER JOIN rentas r ON v.id = r.id_vehiculo
WHERE r.fecha_inicio >= date('now', '-1 year')
GROUP BY v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad'''

result1 = conn.execute(query).fetchall()

# Calcular el tiempo promedio de renta para cada vehículo
query = '''SELECT r.id_vehiculo, AVG(julianday(r.fecha_fin) - julianday(r.fecha_inicio)) AS duracion_promedio
FROM rentas r
GROUP BY r.id_vehiculo'''

result2 = conn.execute(query).fetchall()

# Agregar los datos a una tabla agregada
query = '''SELECT v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad, COUNT(*) AS num_rentas, AVG(julianday(r.fecha_fin) - julianday(r.fecha_inicio)) AS duracion_promedio
FROM vehiculos v
INNER JOIN rentas r ON v.id = r.id_vehiculo
WHERE r.fecha_inicio >= date('now', '-1 year')
GROUP BY v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad'''

result3 = conn.execute(query).fetchall()

# Utilizar esta tabla agregada para realizar análisis con diversas dimensiones
query = '''SELECT marca, modelo, anio, tipo, capacidad, num_rentas, duracion_promedio
FROM (
    SELECT v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad, COUNT(*) AS num_rentas, AVG(julianday(r.fecha_fin) - julianday(r.fecha_inicio)) AS duracion_promedio
    FROM vehiculos v
    INNER JOIN rentas r ON v.id = r.id_vehiculo
    WHERE r.fecha_inicio >= date('now', '-1 year')
    GROUP BY v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad
) AS tabla_agregada
ORDER BY num_rentas DESC, duracion_promedio ASC'''

result4 = conn.execute(query).fetchall()

# Cerrar la conexión a la base de datos
conn.close()

## Creating some fake data 

In [38]:
import pandas as pd
import sqlite3
import random
from faker import Faker

# Crear la conexión a la base de datos
conn = sqlite3.connect('renta_vehiculos.db')

# Crear un objeto Faker para generar datos aleatorios
fake = Faker()

# Generar datos para la tabla de vehículos
vehiculos_data = []
for i in range(1, 101):
    vehiculos_data.append({
        'id_vehiculo': i,
        'marca': fake.company(),
        'modelo':  fake.word(),
        'anio': random.randint(2015, 2023),
        'tipo': random.choice(['Sedan', 'Pickup', 'SUV']),
        'capacidad': random.randint(2, 8),
        'color': fake.color_name(),
        'kilometraje': random.randint(10000, 100000),
        'estado': random.choice(['Disponible', 'En mantenimiento', 'En renta'])
    })

# Generar datos para la tabla de puntos de renta
puntos_renta_data = []
for i in range(1, 11):
    puntos_renta_data.append({
        'id_punto': i,
        'nombre': fake.company(),
        'direccion': fake.address(),
        'telefono': fake.phone_number(),
        'horario_atencion': fake.time(pattern='%H:%M:%S')
    })

# Generar datos para la tabla de clientes
clientes_data = []
for i in range(1, 101):
    clientes_data.append({
        'id_cliente': i,
        'nombre': fake.name(),
        'correo_electronico': fake.email(),
        'telefono': fake.phone_number(),
        'direccion': fake.address()
    })

# Generar datos para la tabla de rentas
rentas_data = []
for i in range(1, 1001):
    rentas_data.append({
        'id_renta': i,
        'id_vehiculo': random.randint(1, 100),
        'id_punto_salida': random.randint(1, 10),
        'id_punto_retorno': random.randint(1, 10),
        'id_cliente': random.randint(1, 100),
        'fecha_inicio': fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d %H:%M:%S'),
        'fecha_fin': fake.date_time_between(start_date='now', end_date='+1y').strftime('%Y-%m-%d %H:%M:%S'),
        'costo_total': random.randint(500, 5000)
    })

# Convertir los datos en DataFrames de pandas
vehiculos_df = pd.DataFrame(vehiculos_data)
puntos_renta_df = pd.DataFrame(puntos_renta_data)
clientes_df = pd.DataFrame(clientes_data)
rentas_df = pd.DataFrame(rentas_data)

# Insertar los datos en las tablas de la base de datos
vehiculos_df.to_sql('vehiculos', conn, if_exists='replace', index=False)
puntos_renta_df.to_sql('puntos_renta', conn, if_exists='replace', index=False)
clientes_df.to_sql('clientes', conn, if_exists='replace', index=False)
rentas_df.to_sql('rentas', conn, if_exists='replace', index=False)

# Cerrar la conexión a la base de datos
conn.close()

In [39]:
import sqlite3

# establecer la ruta de la base de datos
db_file = 'renta_vehiculos.db'

# conectarse a la base de datos
conn = sqlite3.connect(db_file)

# obtener una lista de todas las tablas en la base de datos
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# eliminar cada tabla
for table_name in tables:
    table_name = table_name[0]
    cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
    print(f"La tabla {table_name} ha sido eliminada.")

# confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()

La tabla vehiculos ha sido eliminada.
La tabla puntos_renta ha sido eliminada.
La tabla clientes ha sido eliminada.
La tabla rentas ha sido eliminada.


In [40]:
vehiculos_df 

,id_vehiculo,marca,modelo,anio,tipo,capacidad,color,kilometraje,estado
0,1,Mitchell-Herrera,head,2018,Sedan,3,PaleGreen,97957,En renta
1,2,Sims-Sims,like,2018,SUV,2,LightCyan,22539,Disponible
2,3,Liu-Sosa,lot,2018,Pickup,7,PaleGreen,81031,Disponible
3,4,"Chang, Sullivan and Eaton",process,2018,Sedan,8,DarkGreen,55760,Disponible
4,5,Valdez PLC,account,2020,Sedan,4,LavenderBlush,26164,En renta
...,...,...,...,...,...,...,...,...,...
95,96,Anderson-Middleton,arm,2018,Pickup,2,MistyRose,88150,En renta
96,97,Ortiz-Thompson,local,2021,Sedan,6,Sienna,75490,En renta
97,98,Henderson PLC,performance,2019,SUV,8,PowderBlue,64846,En mantenimiento
98,99,"Holden, Baxter and Douglas",simple,2023,Sedan,6,IndianRed,64072,En mantenimiento


In [41]:
puntos_renta_df

,id_punto,nombre,direccion,telefono,horario_atencion
0,1,Gill PLC,"396 Hoffman Mountain Apt. 500\nSharonland, NM ...",805-467-1713,21:52:46
1,2,Welch-Perry,"39026 Ashley Inlet\nEast Danielville, VT 39835",001-645-543-6215x170,07:04:04
2,3,Armstrong PLC,Unit 4850 Box 0440\nDPO AA 84506,+1-893-879-8501x638,14:35:51
3,4,Gardner-Collier,"1530 Matthew Extensions\nNorrisburgh, GU 46447",001-789-384-9452,20:19:22
4,5,"Navarro, Brown and Whitaker","577 Curry Flat\nWest Brandonside, OR 95378",001-398-269-2865x7750,21:44:24
5,6,English-Thomas,"840 Morgan Highway Suite 358\nNorth Alicia, NJ...",322-383-6329,18:34:27
6,7,"Pacheco, Smith and Walsh","144 Hall Ridge\nRebekahbury, KS 32578",622-599-9008x6877,03:47:54
7,8,"White, Torres and Humphrey",89943 Ramirez Drive Suite 706\nNew Brittanyshi...,7966742561,02:27:48
8,9,Edwards Ltd,"7085 David Walks Suite 190\nLake Tammy, CT 42411",(763)677-1777x519,20:36:20
9,10,"Bailey, Holt and Chaney","PSC 5408, Box 3006\nAPO AE 58153",+1-520-531-8768x11516,06:34:44


In [42]:
clientes_df

,id_cliente,nombre,correo_electronico,telefono,direccion
0,1,Lori Thomas,caitlinbrown@example.org,814.646.8296x205,"385 Newman Walk\nKristenport, VI 25310"
1,2,Chad Smith,bishopfrancis@example.org,(519)422-1414x9142,"281 Allen Centers\nNew Matthewburgh, WI 17236"
2,3,Samantha Burke,alvarezjustin@example.net,+1-284-998-1750x601,"613 Amy Hill Suite 013\nNorth Brittanyfurt, MI..."
3,4,Desiree Petersen,jenniferbarber@example.com,(905)594-6696x5699,"15681 Jonathan Street\nEast Ritaport, MT 17448"
4,5,John Hall,christopher58@example.org,(400)267-2532x47923,"8659 Sarah Loop Suite 739\nLake Jenniferland, ..."
...,...,...,...,...,...
95,96,Brendan Butler,brianyates@example.org,939.461.7998x98983,USCGC Watkins\nFPO AP 30784
96,97,Mrs. Sara Kim MD,jeffrey40@example.org,+1-533-332-0367,USNV Bowers\nFPO AA 59095
97,98,John Miller,dwaynecole@example.org,001-882-327-4407x2084,"25664 Abigail Port\nJulieview, KY 77583"
98,99,Michael Anderson,kristen71@example.net,2349386432,903 Kimberly Drive Suite 093\nEast Crystalstad...


In [43]:
rentas_df

,id_renta,id_vehiculo,id_punto_salida,id_punto_retorno,id_cliente,fecha_inicio,fecha_fin,costo_total
0,1,37,7,7,85,2022-08-09 14:21:01,2024-07-07 11:51:37,4732
1,2,74,10,10,55,2022-07-22 13:39:18,2024-03-19 08:09:06,1488
2,3,41,5,10,4,2022-10-24 06:57:14,2023-11-26 09:04:19,2446
3,4,2,5,8,74,2022-12-05 20:00:56,2024-03-11 22:46:03,3930
4,5,16,10,1,48,2022-08-29 10:56:35,2024-02-21 10:01:43,548
...,...,...,...,...,...,...,...,...
995,996,59,7,3,64,2023-03-25 23:58:12,2024-01-01 14:52:02,620
996,997,89,9,10,75,2022-12-17 17:17:45,2023-08-15 19:05:36,3869
997,998,9,2,9,57,2023-05-29 23:48:18,2024-04-30 17:01:28,1183
998,999,91,7,9,10,2023-05-09 02:46:40,2023-10-09 13:53:16,1003


## Generate some csv files  for simulate a etl pipeline

In [44]:
vehiculos_df.to_csv('vehiculos.csv', encoding='utf-8', index=False)

In [45]:
puntos_renta_df.to_csv('puntos_renta.csv', encoding='utf-8', index=False)

In [46]:
clientes_df.to_csv('clientes.csv', encoding='utf-8', index=False)

In [47]:
rentas_df.to_csv('rentas.csv', encoding='utf-8', index=False)

## Generate DDL 

In [53]:
import sqlite3

# Crear una conexión a la base de datos
conn = sqlite3.connect('renta_vehiculos.db')

# Obtener el DDL para cada tabla en la base de datos
cursor = conn.cursor()
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

# Mostrar el DDL de cada tabla
for table in tables:
    print(table[0])
    print("\n")

# Cerrar la conexión a la base de datos
conn.close()

CREATE TABLE vehiculos (
    id INTEGER PRIMARY KEY,
    marca TEXT,
    modelo TEXT,
    anio INTEGER,
    tipo TEXT,
    capacidad INTEGER,
    color TEXT,
    kilometraje REAL,
    estado TEXT
)


CREATE TABLE puntos_renta (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    direccion TEXT,
    telefono TEXT,
    horario_atencion TEXT
)


CREATE TABLE clientes (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    correo_electronico TEXT,
    telefono TEXT,
    direccion TEXT
)


CREATE TABLE rentas (
    id INTEGER PRIMARY KEY,
    id_vehiculo INTEGER,
    id_punto_salida INTEGER,
    id_punto_retorno INTEGER,
    id_cliente INTEGER,
    fecha_inicio TEXT,
    fecha_fin TEXT,
    costo_total REAL,
    duracion_renta TEXT,
    FOREIGN KEY (id_vehiculo) REFERENCES vehiculos(id_vehiculo),
    FOREIGN KEY (id_punto_salida) REFERENCES puntos_renta(id_punto),
    FOREIGN KEY (id_punto_retorno) REFERENCES puntos_renta(id_punto),
    FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente)
)




In [54]:
import pandas as pd
import sqlite3
import logging

class RentasETL:

    def __init__(self, vehiculos_file, puntos_renta_file, clientes_file, rentas_file):
        self.vehiculos_file = vehiculos_file
        self.puntos_renta_file = puntos_renta_file
        self.clientes_file = clientes_file
        self.rentas_file = rentas_file
        self.conn = sqlite3.connect('renta_vehiculos.db')
        self.vehiculos_inserted_ids = []
        self.puntos_renta_inserted_ids = []
        self.clientes_inserted_ids = []
        self.rentas_inserted_ids = []
        logging.basicConfig(filename='rentas_etl.log', level=logging.INFO)

    def extract_vehiculos(self):
        vehiculos_df = pd.read_csv(self.vehiculos_file)
        vehiculos_df = vehiculos_df.rename(columns={
            'id_vehiculo': 'id',
            'marca': 'marca',
            'modelo': 'modelo',
            'anio': 'anio',
            'tipo': 'tipo',
            'capacidad': 'capacidad',
            'color': 'color',
            'kilometraje': 'kilometraje',
            'estado': 'estado'
        })
        vehiculos_df.name = 'vehiculos'

        # Verificar si los datos ya existen en la tabla vehiculos antes de cargarlos
        vehiculos_df = vehiculos_df.loc[~vehiculos_df['id'].isin(self.vehiculos_inserted_ids)]
        count = 0  # Inicializar contador
        for index, row in vehiculos_df.iterrows():
            id = row['id']
            cursor = self.conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM vehiculos WHERE id = {id}")
            result = cursor.fetchone()
            if result[0] > 0:
                vehiculos_df = vehiculos_df.drop(index)
                logging.info(f"Registro duplicado en la tabla vehiculos: {row}")
            else:
                logging.info(f"Insertando registro en la tabla vehiculos: {row}")
                count += 1  # Incrementar contador en uno

        print(f"Se insertaron {count} nuevos registros en la tabla vehiculos")

        return vehiculos_df

    def extract_puntos_renta(self):
        puntos_renta_df = pd.read_csv(self.puntos_renta_file)
        puntos_renta_df = puntos_renta_df.rename(columns={
            'id_punto': 'id',
            'nombre': 'nombre',
            'direccion': 'direccion',
            'telefono': 'telefono',
            'horario_atencion': 'horario_atencion'
        })


        # Verificar si los datos ya existen en la tabla puntos_renta antes de cargarlos
        puntos_renta_df = puntos_renta_df.loc[~puntos_renta_df['id'].isin(self.puntos_renta_inserted_ids)]
        count = 0  # Inicializar contador
        for index, row in puntos_renta_df.iterrows():
            id = row['id']
            cursor = self.conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM  puntos_renta WHERE id = {id}")
            result = cursor.fetchone()
            if result[0] > 0:
                puntos_renta_df = puntos_renta_df.drop(index)
                logging.info(f"Registro duplicado en la tabla puntos_renta: {row}")
            else:
                logging.info(f"Insertando registro en la tabla puntos_renta: {row}")
                count += 1  # Incrementar contador en uno

        print(f"Se insertaron {count} nuevos registros en la tabla puntos_renta")
        return puntos_renta_df

    def extract_clientes(self):
        clientes_df = pd.read_csv(self.clientes_file)
        clientes_df = clientes_df.rename(columns={
            'id_cliente': 'id',
            'nombre': 'nombre',
            'correo_electronico': 'correo_electronico',
            'telefono': 'telefono',
            'direccion': 'direccion'
        })
        clientes_df.name = 'clientes'

        # Verificar si los datos ya existen en la tabla clientes antes de cargarlos
        clientes_df = clientes_df.loc[~clientes_df['id'].isin(self.clientes_inserted_ids)]

        count = 0  # Inicializar contador

        for index, row in clientes_df.iterrows():
            id = row['id']
            cursor = self.conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM clientes WHERE id = {id}")
            result = cursor.fetchone()
            if result[0] > 0:
                clientes_df = clientes_df.drop(index)
                logging.info(f"Registro duplicado en la tabla clientes: {row}")
            else:
                logging.info(f"Insertando registro en la tabla clientes: {row}")
                count += 1  # Incrementar contador en uno

        print(f"Se insertaron {count} nuevos registros en la tabla clientes")

        return clientes_df

    def extract_rentas(self):
        rentas_df = pd.read_csv(self.rentas_file)
        rentas_df = rentas_df.rename(columns={
            'id_renta': 'id',
            'id_vehiculo': 'id_vehiculo',
            'id_punto_salida': 'id_punto_salida',
            'id_punto_retorno': 'id_punto_retorno',
            'id_cliente': 'id_cliente',
            'fecha_inicio': 'fecha_inicio',
            'fecha_fin': 'fecha_fin',
            'costo_total': 'costo_total'
        })
        rentas_df.name = 'rentas'

        # Agregar columna de duración de la renta en días
        rentas_df['duracion_renta'] = (pd.to_datetime(rentas_df['fecha_fin']) - pd.to_datetime(rentas_df['fecha_inicio'])).dt.days

        # Eliminar registros con duración negativa o cero
        rentas_df = rentas_df.loc[rentas_df['duracion_renta'] > 0]
   

        # Verificar si los datos ya existen en la tabla rentas antes de cargarlos
        rentas_df = rentas_df.loc[~rentas_df['id'].isin(self.rentas_inserted_ids)]
        count = 0  # Inicializar contador
        for index, row in rentas_df.iterrows():
            id = row['id']
            cursor = self.conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM rentas WHERE id = {id}")
            result = cursor.fetchone()
            if result[0] > 0:
                rentas_df = rentas_df.drop(index)
                logging.info(f"Registro duplicado en la tabla rentas: {row}")
            else:
                logging.info(f"Insertando registro en la tabla rentas: {row}")
                count += 1  # Incrementar contador en uno

        print(f"Se insertaron {count} nuevos registros en la tabla rentas")
        return rentas_df

    def transform_vehiculos(self, vehiculos_df):
        vehiculos_df['marca'] = vehiculos_df['marca'].str.capitalize()
        vehiculos_df['modelo'] = vehiculos_df['modelo'].str.capitalize()
        vehiculos_df['tipo'] = vehiculos_df['tipo'].str.capitalize()
        vehiculos_df['color'] = vehiculos_df['color'].str.capitalize()
        return vehiculos_df

    def transform_puntos_renta(self, puntos_renta_df):
        puntos_renta_df['nombre'] = puntos_renta_df['nombre'].str.title()
        puntos_renta_df['direccion'] = puntos_renta_df['direccion'].str.title()
        return puntos_renta_df

    def transform_clientes(self, clientes_df):
        clientes_df['nombre'] = clientes_df['nombre'].str.title()
        clientes_df['correo_electronico'] = clientes_df['correo_electronico'].str.lower()
        clientes_df['direccion'] = clientes_df['direccion'].str.title()
        return clientes_df

    def load_vehiculos(self, vehiculos_df):
        try:
            vehiculos_df.to_sql('vehiculos', self.conn, if_exists='append', index=False)
            self.vehiculos_inserted_ids.extend(vehiculos_df['id'].tolist())
        except Exception as e:
            logging.error(f"Error al cargar la tabla vehiculos: {str(e)}")

    def load_puntos_renta(self, puntos_renta_df):
        try:
            puntos_renta_df.to_sql('puntos_renta', self.conn, if_exists='append', index=False)
            self.puntos_renta_inserted_ids.extend(puntos_renta_df['id'].tolist())
        except Exception as e:
            logging.error(f"Error al cargar la tabla puntos_renta: {str(e)}")

    def load_clientes(self, clientes_df):
        try:
            clientes_df.to_sql('clientes', self.conn, if_exists='append', index=False)
            self.clientes_inserted_ids.extend(clientes_df['id'].tolist())
        except Exception as e:
            logging.error(f"Error al cargar la tabla clientes: {str(e)}")

    def load_rentas(self, rentas_df):
        try:
            rentas_df.to_sql('rentas', self.conn, if_exists='append', index=False)
            self.rentas_inserted_ids.extend(rentas_df['id'].tolist())
        except Exception as e:
            logging.error(f"Error al cargar la tabla rentas: {str(e)}")

    def run(self):
        print("Init the Etl")
        vehiculos_df = self.extract_vehiculos()
        puntos_renta_df = self.extract_puntos_renta()
        clientes_df = self.extract_clientes()
        rentas_df = self.extract_rentas()

        vehiculos_df = self.transform_vehiculos(vehiculos_df)
        puntos_renta_df = self.transform_puntos_renta(puntos_renta_df)
        clientes_df = self.transform_clientes(clientes_df)

        self.load_vehiculos(vehiculos_df)
        self.load_puntos_renta(puntos_renta_df)
        self.load_clientes(clientes_df)
        self.load_rentas(rentas_df)

        self.conn.commit()
        self.conn.close()

In [55]:
etl = RentasETL('vehiculos.csv', 'puntos_renta.csv', 'clientes.csv', 'rentas.csv')

In [56]:
etl.run()

Init the Etl
Se insertaron 100 nuevos registros en la tabla vehiculos
Se insertaron 10 nuevos registros en la tabla puntos_renta
Se insertaron 100 nuevos registros en la tabla clientes
Se insertaron 1000 nuevos registros en la tabla rentas


In [57]:
import pandas_profiling
from pandas_profiling.utils.cache import cache_file

/var/folders/8y/r6dqnc1n58zgz0w3clll7dxm0000gn/T/ipykernel_27881/1577145084.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [59]:
## Ahora corremos el reporte
report = rentas_df.profile_report(sort='None', html={'style':{'full_width':True}})

In [60]:
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

ValueError: "sort" should be "ascending", "descending" or None.